# Olap analysis

This notebook contains the code dealing with olap analysis: cube design, table creation and schema loading.

The first thing to do is the creation of a schema for the OLAP analysis and the connection

In [ ]:
require 'pg'

conn = PG.connect(dbname: 'mimic3_demo', host: 'postgres', user: 'postgres', password: 'password')
conn.exec('CREATE SCHEMA olap')
conn.close

PG::DuplicateSchema: ERROR:  schema "olap" already exists


In [2]:
require 'sequel'

DB_MIMIC = Sequel.connect(adapter: 'postgres',
                          host: 'postgres',
                          user: 'postgres',
                          password: 'password',
                          database: 'mimic3_demo',
                          search_path: ['mimiciii']
                          )

DB_OLAP = Sequel.connect(adapter: 'postgres',
                          host: 'postgres',
                          user: 'postgres',
                          password: 'password',
                          database: 'mimic3_demo',
                          search_path: ['olap']
                          )

load '../etl/model.rb'
nil

## Schema 0 - Test: Admission

![Cube 0](img/Cube0.png)

#### Time dimension table

In [ ]:
# Creation
DB_OLAP.drop_table?(:admissions_time)
DB_OLAP.create_table(:admissions_time) do
  DateTime :datetime, primary_key: true
  String :day
  String :month
  String :year
end

# Population
Admission.all.uniq.each do |admission|
  time = admission.admittime
  year = time.year
  month = time.month
  day = time.day
  DB_OLAP[:admissions_time].insert(
    datetime: time,
    day: "#{year}-#{month}-#{day}",
    month: "#{year}-#{month}",
    year: year.to_s
  )
end

nil

#### Fact table

In [ ]:
# Creation
DB_OLAP.drop_table?(:admissions)
DB_OLAP.create_table(:admissions) do
  Integer :id, primary_key: true
  DateTime :datetime
  String :type
  String :insurance
  String :location
  Integer :death
end

# Population
Admission.all.uniq.each do |admission|
  DB_OLAP[:admissions].insert(
    id: admission.row_id,
    datetime: admission.admittime,
    type: admission.admission_type,
    insurance: admission.insurance,
    location: admission.admission_location,
    death: admission.hospital_expire_flag
  )
end

nil

## Schema 1: Intensive care unit stay

![Cube 1](img/Cube1.png)

#### Time dimension table

In [4]:
# Creation
DB_OLAP.drop_table?(:icu_stay_time)
DB_OLAP.create_table(:icu_stay_time) do
  DateTime :datetime, primary_key: true
  String :day
  String :month
  String :year
end

# Population
Icustay.all.uniq.each do |icustay|
  time = icustay.intime
  year = time.year
  month = time.month
  day = time.day
  DB_OLAP[:icu_stay_time].insert(
    datetime: time,
    day: "#{year}-#{month}-#{day}",
    month: "#{year}-#{month}",
    year: year.to_s
  )
end

nil

#### Patient age dimension table

In [73]:
def age_in_completed_years (bd, d)
    # Difference in years, less one if you have not had a birthday this year.
    a = d.year - bd.year
    a = a - 1 if (
         bd.month >  d.month or 
        (bd.month >= d.month and bd.day > d.day)
    )
    a
end

:age_in_completed_years

In [76]:
# Creation
DB_OLAP.drop_table?(:icu_stay_pat_age)
DB_OLAP.create_table(:icu_stay_pat_age) do
  Integer :age_id, primary_key: true
  String :age
  String :decade
end

# Population
Icustay.all.uniq.each do |icustay|
  patient = icustay.patient
  age = age_in_completed_years(patient.dob, icustay.intime)
  decade = age / 10
  decade_str = "#{decade}0 - #{decade}9"  
  if DB_OLAP[:icu_stay_pat_age].where(age_id: age).empty?
    DB_OLAP[:icu_stay_pat_age].insert(
      age_id: age,
      age: age.to_s,
      decade: decade_str
    )
  end
end

nil

In [8]:
def find_cat (icd9)
  if icd9.match /^(V|E)[0-9]*$/
    return "E and V codes: external causes of injury and supplemental classification"
  end
  icd9 = icd9[0..2].to_i
  if icd9 < 140
    return "001-139: infectious and parasitic diseases"
  end
  if icd9 < 240
    return "140-239: neoplasms"
  end
  if icd9 < 280
    return "240-279: endocrine, nutritional and metabolic diseases, and immunity disorders"
  end
  if icd9 < 290
    return "280-289: diseases of the blood and blood-forming organs"
  end
  if icd9 < 320
    return "290-319: mental disorders"
  end
  if icd9 < 390
    return "320-389: diseases of the nervous system and sense organs"
  end
  if icd9 < 460
    return "390-459: diseases of the circulatory system"
  end
  if icd9 < 520
    return "460-519: diseases of the respiratory system"
  end
  if icd9 < 580
    return "520-579: diseases of the digestive system"
  end
  if icd9 < 630
    return "580-629: diseases of the genitourinary system"
  end
  if icd9 < 680
    return "630-679: complications of pregnancy, childbirth, and the puerperium"
  end
  if icd9 < 710
    return "680-709: diseases of the skin and subcutaneous tissue"
  end
  if icd9 < 740
    return "710-739: diseases of the musculoskeletal system and connective tissue"
  end
  if icd9 < 760
    return "740-759: congenital anomalies"
  end
  if icd9 < 780
    return "760-779: certain conditions originating in the perinatal period"
  end
  if icd9 < 800
    return "780-799: symptoms, signs, and ill-defined conditions"
  end
  if icd9 < 1000
    return "800-999: injury and poisoning"
  end
  return ""
end 
    

:find_cat

#### Diagnose dimension table

In [ ]:
#Creation
# Creation
DB_OLAP.drop_table?(:icu_stay_diagnose)
DB_OLAP.create_table(:icu_stay_diagnose) do
  String :icd9_code, primary_key: true
  String :icd9_cat
end

# Population
Icustay.all.uniq.each do |icustay|
  admission = icustay.admission
  admission.diagnoses_icd.each do |diagnose|
    icd9 = diagnose.icd9_code
    if DB_OLAP[:icu_stay_diagnose].where(icd9_code: icd9).empty?
      DB_OLAP[:icu_stay_diagnose],insert(
        icd9_code: icd9,
        icd9_cat: find_cat(icd9)
      )
    end
  end
end

nil

## TESTS

In [48]:
DB_OLAP[:icu_stay_pat_age].where(age_id: 5).empty?

true

In [52]:
[1, 2, 3].methods.grep(/\?/)

[:include?, :any?, :empty?, :eql?, :frozen?, :all?, :one?, :none?, :member?, :instance_variable_defined?, :instance_of?, :kind_of?, :is_a?, :respond_to?, :nil?, :tainted?, :untrusted?, :equal?]

In [77]:
a = Icustay.first.intime
b = Icustay.first.patient.dob
age_in_completed_years(b, a)

70

In [87]:
a = "453"
if a =~ /^(V|E)[0-9]*/
  puts 'yes'
else
  puts 'no'
end

no
